In [116]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

pd.set_option('display.max_columns', None)
%matplotlib inline

In [117]:
df = pd.read_csv('data/diabetic_data.csv')
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,?,Pediatrics-Endocrinology,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,?,?,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,?,?,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,?,?,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,?,?,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [118]:
df.shape

(101766, 50)

In [119]:
# pandas_profiling.ProfileReport(df)

In [120]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [121]:
# look at a count of missing (?) values in the dataset
for col in df.columns:
    if df[col].dtype == object:
        if df[col][df[col] == '?'].count() != 0:
            print(col,df[col][df[col] == '?'].count(),(df[col][df[col] == '?'].count()/101766)*100)
            
# gender was coded differently so we use a custom count for this one
print('gender', df['gender'][df['gender'] == 'Unknown/Invalid'].count(),
      (df['gender'][df['gender'] == 'Unknown/Invalid'].count()/101766)*100)

race 2273 2.2335554114340743
weight 98569 96.85847925633315
payer_code 40256 39.5574160328597
medical_specialty 49949 49.08220820313268
diag_1 21 0.02063557573256294
diag_2 358 0.3517874339170253
diag_3 1423 1.398305917497003
gender 3 0.002947939390366134


Weight is missing in over 96% of records in the dataset. Probably should drop this entire column since there are very few records that actually have data.

Also, Payer Code and Medical Specialty of the physician are missing 40-50% of the time. I am going to drop these columns from the dataset as well, although I may revisit this in the future since it would be interesting to see the effect that these values have on readmissions.

In [122]:
# dropping weight, payer_code, and medical_specialty from the dataset
df.drop(['weight', 'payer_code', 'medical_specialty'], axis = 1, inplace=True)

# check the column count
df.shape

(101766, 47)

In [123]:
# dropping records where gender is missing
df = df[df.gender != 'Unknown/Invalid']

# check the row count
df.shape

(101763, 47)

In [124]:
def diag_missing_label(row):
    """This function will return a 1 if all diag attributes are missing"""
    
    if row['diag_1'] == '?' and row['diag_2'] == '?' and row['diag_3'] == '?':
        return 1
    else:
        return 0

In [125]:
# apply function to return a 1 if all diag attributes
df['diag_missing_label'] = df.apply(lambda row: diag_missing_label(row), axis=1)

In [126]:
df['diag_missing_label'].value_counts()

0    101762
1         1
Name: diag_missing_label, dtype: int64

In [127]:
# drop records where all diag attributes are missing
df = df[df.diag_missing_label != 1]

df.shape

(101762, 48)

In [128]:
df.discharge_disposition_id.value_counts()

1     60232
3     13954
6     12902
18     3691
2      2128
22     1992
11     1642
5      1184
25      989
4       815
7       623
23      412
13      398
14      372
28      139
8       108
15       63
24       48
9        21
17       14
16       11
19        8
10        6
27        5
12        3
20        2
Name: discharge_disposition_id, dtype: int64

In [129]:
# drop records where discharge disposition = 11 (patient died)
df = df[df.discharge_disposition_id != 11]

df.shape

(100120, 48)

In [130]:
# are there multiple records per patient id?
df.patient_nbr.nunique()

70442

In [131]:
# drop duplicates of patient id to match assumption of logistic regression model
df.drop_duplicates('patient_nbr', inplace=True)

df.shape

(70442, 48)

In [132]:
def get_unique_values(df):
    """This function will print out number of unique values for each column in the dataset"""
    
    for col in df.columns:
        print(str(col), df[col].nunique())

In [133]:
get_unique_values(df)

encounter_id 70442
patient_nbr 70442
race 6
gender 2
age 10
admission_type_id 8
discharge_disposition_id 25
admission_source_id 17
time_in_hospital 14
num_lab_procedures 116
num_procedures 7
num_medications 75
number_outpatient 33
number_emergency 18
number_inpatient 13
diag_1 696
diag_2 725
diag_3 758
number_diagnoses 16
max_glu_serum 4
A1Cresult 4
metformin 4
repaglinide 4
nateglinide 4
chlorpropamide 4
glimepiride 4
acetohexamide 2
glipizide 4
glyburide 4
tolbutamide 2
pioglitazone 4
rosiglitazone 4
acarbose 3
miglitol 4
troglitazone 2
tolazamide 2
examide 1
citoglipton 1
insulin 4
glyburide-metformin 4
glipizide-metformin 2
glimepiride-pioglitazone 1
metformin-rosiglitazone 2
metformin-pioglitazone 2
change 2
diabetesMed 2
readmitted 3
diag_missing_label 1


There are four columns that only have 1 value for all records. These cannot provide any reliable information for predicting readmission, so let's drop the following columns:

examide, citoglipton, glimepiride-pioglitazone, diag_missing_label (which was created above)

In [134]:
# drop the four columns that only have 1 value for all records
df.drop(['examide', 'citoglipton', 'glimepiride-pioglitazone', 'diag_missing_label'], axis=1, inplace=True)

df.shape

(70442, 44)

### Feature Engineering

In [135]:
# create a column for service utilization
df['service_utilization'] = df.number_outpatient + df.number_emergency + df.number_inpatient

In [136]:
df.sort_values(by='service_utilization', ascending=False).head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization
32661,103948200,42466248,Caucasian,Female,[30-40),1,1,7,3,45,0,14,3,42,4,250.11,346,311,5,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Yes,>30,49
64743,180134730,85551867,Caucasian,Male,[80-90),1,6,7,3,9,0,18,42,0,0,428,427,799,9,None,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30,42
86135,273615240,90847035,Caucasian,Male,[40-50),1,1,7,1,58,1,8,0,37,0,780,304,722,9,None,None,Steady,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,37
67249,188678112,88598556,Caucasian,Female,[60-70),5,2,1,1,5,1,25,36,0,0,410,599,427,9,None,>8,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,NO,36
32601,103784028,23491422,Caucasian,Male,[80-90),5,6,17,6,19,2,14,35,0,0,426,428,425,7,>200,None,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,35


Service utilization looks to be higher for older patients and therefore is likely associated with more readmissions. This could be a good predictor of readmissions, although hard to tell if these are because of diabetes or because of other complications due to older age.

In [137]:
meds = ['metformin',
'repaglinide',
'nateglinide',
'chlorpropamide',
'glimepiride',
'acetohexamide',
'glipizide',
'glyburide',
'tolbutamide',
'pioglitazone',
'rosiglitazone',
'acarbose',
'miglitol',
'troglitazone',
'tolazamide',
'insulin',
'glyburide-metformin',
'glipizide-metformin',
'metformin-rosiglitazone',
'metformin-pioglitazone',]



df['num_diabetes_meds'] = 0
for med in meds:
    colname = str(med) + 'temp'
    df[colname] = df[med].apply(lambda x: 0 if x == 'No' else 1)
    df['num_diabetes_meds'] = df['num_diabetes_meds'] + df[colname]
    del(df[colname])

In [138]:
df.num_diabetes_meds.value_counts()

1    31372
0    16829
2    15503
3     5680
4     1004
5       50
6        4
Name: num_diabetes_meds, dtype: int64

Most patients were either not on medications or were taking one medication during their visit. However, there are still over 1,000 patients that were taking two or more medications. I will be using the number of medications as one of the features to predict readmissions.

In [139]:
df.diabetesMed.value_counts()

Yes    53613
No     16829
Name: diabetesMed, dtype: int64

In [140]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,0,0
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,>30,0,1
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,3,1
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,NO,0,1
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,0,2


Now looking at diagnoses codes, it appears I need to clean the data. There are several question marks ('?') in these columns, which probably represent null or missing values.

In [141]:
print("There are: " + str(len(df[df["diag_1"] == "?"])) + " rows where diag_1 = ?")
print("There are: " + str(len(df[df["diag_2"] == "?"])) + " rows where diag_2 = ?")
print("There are: " + str(len(df[df["diag_3"] == "?"])) + " rows wehre diag_3 = ?")

There are: 10 rows where diag_1 = ?
There are: 293 rows where diag_2 = ?
There are: 1224 rows wehre diag_3 = ?


Because there are several rows that have question marks ('?') indicating missing values, I will keep these records in the dataset. I will code them separately from the rest of the data - perhaps using a '-1' to represent 'missing.'

In [142]:
# create copies of the 3 diagnosis columns
df['level1_diag'] = df['diag_1']
df['level2_diag'] = df['diag_2']
df['level3_diag'] = df['diag_3']

# deal with the missing values first
df['level1_diag'] = df['level1_diag'].replace('?', -1)
df['level2_diag'] = df['level2_diag'].replace('?', -1)
df['level3_diag'] = df['level3_diag'].replace('?', -1)

# when V or E is at beginning of diagnosis code, recode as 'Other'
df.loc[df['diag_1'].str.contains('V'), ['level1_diag']] = 0
df.loc[df['diag_1'].str.contains('E'), ['level1_diag']] = 0
df.loc[df['diag_2'].str.contains('V'), ['level2_diag']] = 0
df.loc[df['diag_2'].str.contains('E'), ['level2_diag']] = 0
df.loc[df['diag_3'].str.contains('V'), ['level3_diag']] = 0
df.loc[df['diag_3'].str.contains('E'), ['level3_diag']] = 0

In [143]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,0,0,250.83,-1,-1
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,>30,0,1,276,250.01,255
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,3,1,648,250,0
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,NO,0,1,8,250.43,403
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,0,2,197,157,250


In [144]:
# change new diagnosis columns to int
df['level1_diag'] = df['level1_diag'].astype(float)
df['level2_diag'] = df['level2_diag'].astype(float)
df['level3_diag'] = df['level3_diag'].astype(float)

In [145]:
# level1_diagnosis recoding
for index, row in df.iterrows():
    if (row['level1_diag'] >= 390 and row['level1_diag'] <= 459) or (np.floor(row['level1_diag']) == 785):
        df.loc[index, 'level1_diag'] = 1
    elif (row['level1_diag'] >= 460 and row['level1_diag'] <= 519) or (np.floor(row['level1_diag']) == 786):
        df.loc[index, 'level1_diag'] = 2
    elif (row['level1_diag'] >= 520 and row['level1_diag'] <= 579) or (np.floor(row['level1_diag']) == 787):
        df.loc[index, 'level1_diag'] = 3
    elif np.floor(row['level1_diag']) == 250:
        df.loc[index, 'level1_diag'] = 4
    elif (row['level1_diag'] >= 800 and row['level1_diag'] <= 999):
        df.loc[index, 'level1_diag'] = 5
    elif (row['level1_diag'] >= 710 and row['level1_diag'] <= 739):
        df.loc[index, 'level1_diag'] = 6
    elif (row['level1_diag'] >= 580 and row['level1_diag'] <= 629) or (np.floor(row['level1_diag']) == 788):
        df.loc[index, 'level1_diag'] = 7
    elif (row['level1_diag'] >= 140 and row['level1_diag'] <= 239):
        df.loc[index, 'level1_diag'] = 8
    else:
        df.loc[index, 'level1_diag'] = 0

In [146]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,0,0,4.0,-1.00,-1.0
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,>30,0,1,0.0,250.01,255.0
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,3,1,0.0,250.00,0.0
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,NO,0,1,0.0,250.43,403.0
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,0,2,8.0,157.00,250.0


In [147]:
# repeat recoding step above for levels 2 and 3 diagnosis

# level2_diagnosis recoding
for index, row in df.iterrows():
    if (row['level2_diag'] >= 390 and row['level2_diag'] <= 459) or (np.floor(row['level2_diag']) == 785):
        df.loc[index, 'level2_diag'] = 1
    elif (row['level2_diag'] >= 460 and row['level2_diag'] <= 519) or (np.floor(row['level2_diag']) == 786):
        df.loc[index, 'level2_diag'] = 2
    elif (row['level2_diag'] >= 520 and row['level2_diag'] <= 579) or (np.floor(row['level2_diag']) == 787):
        df.loc[index, 'level2_diag'] = 3
    elif np.floor(row['level2_diag']) == 250:
        df.loc[index, 'level2_diag'] = 4
    elif (row['level2_diag'] >= 800 and row['level2_diag'] <= 999):
        df.loc[index, 'level2_diag'] = 5
    elif (row['level2_diag'] >= 710 and row['level2_diag'] <= 739):
        df.loc[index, 'level2_diag'] = 6
    elif (row['level2_diag'] >= 580 and row['level2_diag'] <= 629) or (np.floor(row['level2_diag']) == 788):
        df.loc[index, 'level2_diag'] = 7
    elif (row['level2_diag'] >= 140 and row['level2_diag'] <= 239):
        df.loc[index, 'level2_diag'] = 8
    else:
        df.loc[index, 'level2_diag'] = 0
        
# level3_diagnosis recoding
for index, row in df.iterrows():
    if (row['level3_diag'] >= 390 and row['level3_diag'] <= 459) or (np.floor(row['level3_diag']) == 785):
        df.loc[index, 'level3_diag'] = 1
    elif (row['level3_diag'] >= 460 and row['level3_diag'] <= 519) or (np.floor(row['level3_diag']) == 786):
        df.loc[index, 'level3_diag'] = 2
    elif (row['level3_diag'] >= 520 and row['level3_diag'] <= 579) or (np.floor(row['level3_diag']) == 787):
        df.loc[index, 'level3_diag'] = 3
    elif np.floor(row['level3_diag']) == 250:
        df.loc[index, 'level3_diag'] = 4
    elif (row['level3_diag'] >= 800 and row['level3_diag'] <= 999):
        df.loc[index, 'level3_diag'] = 5
    elif (row['level3_diag'] >= 710 and row['level3_diag'] <= 739):
        df.loc[index, 'level3_diag'] = 6
    elif (row['level3_diag'] >= 580 and row['level3_diag'] <= 629) or (np.floor(row['level3_diag']) == 788):
        df.loc[index, 'level3_diag'] = 7
    elif (row['level3_diag'] >= 140 and row['level3_diag'] <= 239):
        df.loc[index, 'level3_diag'] = 8
    else:
        df.loc[index, 'level3_diag'] = 0

In [148]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO,0,0,4.0,0.0,0.0
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,>30,0,1,0.0,4.0,0.0
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO,3,1,0.0,4.0,0.0
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,Ch,Yes,NO,0,1,0.0,4.0,1.0
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,Ch,Yes,NO,0,2,8.0,8.0,4.0


In [149]:
# check to see that the codes are correct for each diag level
df[['diag_1', 'level1_diag']].head(10).T

,0,1,2,3,4,5,6,7,8,9
diag_1,250.83,276,648,8,197,414,414,428,398,434
level1_diag,4,0,0,0,8,1,1,1,1,1


In [150]:
df[['diag_2', 'level2_diag']].head(10).T

,0,1,2,3,4,5,6,7,8,9
diag_2,?,250.01,250,250.43,157,411,411,492,427,198
level2_diag,0,4,4,4,8,1,1,2,1,8


In [151]:
df[['diag_3', 'level3_diag']].head(10).T

,0,1,2,3,4,5,6,7,8,9
diag_3,?,255,V27,403,250,250,V45,250,38,486
level3_diag,0,0,0,1,4,4,0,4,0,2


Now that we have recoded diagnoses, let's look at other columns that need recoding. In order to generate the model, I need to recode all string variables to binary or integer values.

In [152]:
# let's take a look at the race column
df.race.value_counts()

Caucasian          52672
AfricanAmerican    12693
?                   1920
Hispanic            1506
Other               1160
Asian                491
Name: race, dtype: int64

We see there are a 1,920 Unknown ('?') values for race. I will recode these to be -1, as I did for diagnosis codes above. Also, there are 1,160 Other values, which I will recode as 0.

In [153]:
# recoding race column
df.race = df.race.replace('?', -1)
df.race = df.race.replace('Other', 0)
df.race = df.race.replace('Caucasian', 1)
df.race = df.race.replace('AfricanAmerican', 2)
df.race = df.race.replace('Hispanic', 3)
df.race = df.race.replace('Asian', 4)

In [154]:
# let's take a look at the gender column
df.gender.value_counts()

Female    37475
Male      32967
Name: gender, dtype: int64

In [155]:
# recoding gender column (Male = 1, Female = 0)
df.gender = df.gender.replace('Male', 1)
df.gender = df.gender.replace('Female', 0)

The change and diabetesMed columns are currently text values and I need to recode these to be binary.

In [156]:
# let's take a look at these columns
print("change column:")
print(df.change.value_counts())
print('-----')
print("diabetesMed column:")
print(df.diabetesMed.value_counts())

change column:
No    38778
Ch    31664
Name: change, dtype: int64
-----
diabetesMed column:
Yes    53613
No     16829
Name: diabetesMed, dtype: int64


In [157]:
# recoding change column (Ch = 1, No = 0)
df.change = df.change.replace('Ch', 1)
df.change = df.change.replace('No', 0)

# recoding diabetesMed column (Yes = 1, No = 0)
df.diabetesMed = df.diabetesMed.replace('Yes', 1)
df.diabetesMed = df.diabetesMed.replace('No', 0)

In [158]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
0,2278392,8222157,1,0,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0,0,NO,0,0,4.0,0.0,0.0
1,149190,55629189,1,0,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,1,1,>30,0,1,0.0,4.0,0.0
2,64410,86047875,2,0,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,0,1,NO,3,1,0.0,4.0,0.0
3,500364,82442376,1,1,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,None,None,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,1,1,NO,0,1,0.0,4.0,1.0
4,16680,42519267,1,1,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,None,None,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,1,1,NO,0,2,8.0,8.0,4.0


In [159]:
# let's take a look at the max_glu_serum and A1Cresult variables
print("max_glu_serum:")
print(df.max_glu_serum.value_counts())
print("-----")
print("A1Cresult:")
print(df.A1Cresult.value_counts())

max_glu_serum:
None    67057
Norm     1717
>200      946
>300      722
Name: max_glu_serum, dtype: int64
-----
A1Cresult:
None    57563
>8       6249
Norm     3758
>7       2872
Name: A1Cresult, dtype: int64


In [160]:
# let's recode these to be High, Above Normal, Normal, Unknown
df.max_glu_serum = df.max_glu_serum.replace('>300', 2)
df.max_glu_serum = df.max_glu_serum.replace('>200', 1)
df.max_glu_serum = df.max_glu_serum.replace('Norm', 0)
df.max_glu_serum = df.max_glu_serum.replace('None', -1)

df.A1Cresult = df.A1Cresult.replace('>8', 2)
df.A1Cresult = df.A1Cresult.replace('>7', 1)
df.A1Cresult = df.A1Cresult.replace('Norm', 0)
df.A1Cresult = df.A1Cresult.replace('None', -1)

In [161]:
df.metformin.value_counts()

No        55506
Steady    13663
Up          838
Down        435
Name: metformin, dtype: int64

The medication variables are coded as either No, Steady, Up, or Down. I need to recode these as numeric values as a mask.

In [162]:
# recoding as: No = 0, Steady = 1, Up = 2, Down = -1
# using the same list of med columns from above
for med in meds:
    df[med] = df[med].replace('Up', 2)
    df[med] = df[med].replace('Steady', 1)
    df[med] = df[med].replace('Down', -1)
    df[med] = df[med].replace('No', 0)

In [163]:
# check to see that the recoding is going smoothly
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
0,2278392,8222157,1,0,[0-10),6,25,1,1,41,0,1,0,0,0,250.83,?,?,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NO,0,0,4.0,0.0,0.0
1,149190,55629189,1,0,[10-20),1,1,7,3,59,0,18,0,0,0,276,250.01,255,9,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,1,>30,0,1,0.0,4.0,0.0
2,64410,86047875,2,0,[20-30),1,1,7,2,11,5,13,2,0,1,648,250,V27,6,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NO,3,1,0.0,4.0,0.0
3,500364,82442376,1,1,[30-40),1,1,7,2,44,1,16,0,0,0,8,250.43,403,7,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,1,NO,0,1,0.0,4.0,1.0
4,16680,42519267,1,1,[40-50),1,1,7,1,51,0,8,0,0,0,197,157,250,5,-1,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,NO,0,2,8.0,8.0,4.0


The age column presents some challenges since each record is a range of 10 years. In order to see how age affects readmission, we will want a way to identify an approximate age per patient rather than coding age as a categorical varaible (1 for 0-10, 2 for 10-20, etc). Here, we will take the midpoints of each age range.

In [164]:
# recode age as the midpoint in the range
df.age = df.age.replace('[0-10)', 5)
df.age = df.age.replace('[10-20)', 15)
df.age = df.age.replace('[20-30)', 25)
df.age = df.age.replace('[30-40)', 35)
df.age = df.age.replace('[40-50)', 45)
df.age = df.age.replace('[50-60)', 55)
df.age = df.age.replace('[60-70)', 65)
df.age = df.age.replace('[70-80)', 75)
df.age = df.age.replace('[80-90)', 85)
df.age = df.age.replace('[90-100)', 95)
df.age = df.age.replace('[100-110)', 105)

In [165]:
# check to see if the age recoding worked and see the distibution of ages in the dataset
df.age.value_counts()

75    17889
65    15766
55    12368
85    11270
45     6840
35     2692
95     1808
25     1122
15      534
5       153
Name: age, dtype: int64

In [167]:
df.corr()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,service_utilization,num_diabetes_meds,level1_diag,level2_diag,level3_diag
encounter_id,1.000000,0.502268,-0.032077,0.008878,0.050840,-0.149827,-0.138216,-0.112560,-0.069810,-0.052845,-0.006112,0.056097,0.070823,0.051751,-0.040226,0.257703,-0.128888,0.003860,0.038692,0.007832,0.018072,-0.022523,0.025985,-0.002650,-0.020350,-0.059607,-0.012498,0.045842,-0.044648,-0.009788,-0.000067,-0.010123,-0.017241,0.056672,0.035006,-0.000103,0.008583,0.009258,0.078232,0.040558,0.054295,0.050344,0.022942,0.035262,0.007196
patient_nbr,0.502268,1.000000,-0.030805,0.010191,0.062052,-0.009516,-0.142239,-0.018797,-0.017838,0.005831,-0.024706,0.015700,0.087412,0.057149,0.093112,0.228917,-0.069766,-0.022630,0.008385,0.030517,0.016086,-0.012861,0.013598,-0.003199,-0.001259,-0.040666,-0.001095,0.037268,-0.010480,0.001700,0.003003,-0.002419,-0.012363,0.003154,0.029156,-0.003319,0.011070,-0.001302,0.047660,0.020272,0.124862,0.027464,0.012354,0.014429,0.006070
race,-0.032077,-0.030805,1.000000,-0.047200,-0.116464,-0.033249,0.008626,0.008553,-0.005720,0.004513,-0.038409,-0.052729,-0.025687,0.027697,0.000817,-0.041308,-0.007727,0.022394,-0.001384,0.001897,0.010923,-0.004206,-0.015617,-0.001000,-0.003384,-0.015986,-0.005528,-0.009165,-0.006826,-0.006281,-0.000344,-0.001733,0.000860,-0.006028,-0.003722,-0.002647,-0.017780,-0.001000,-0.007322,-0.012914,-0.008887,-0.015795,0.000677,0.003150,-0.005215
gender,0.008878,0.010191,-0.047200,1.000000,-0.052880,0.011784,-0.021852,0.003336,-0.025157,0.004697,0.066634,-0.012845,-0.014817,-0.020396,-0.001671,-0.002742,-0.000472,0.027997,0.002583,-0.004837,-0.004470,0.000693,-0.003848,-0.003534,0.027125,0.029775,0.001912,0.012257,0.009141,0.008085,0.000170,0.006958,0.004082,0.003548,0.001446,0.004921,0.005681,-0.003534,0.018884,0.018882,-0.019039,0.029738,-0.024394,-0.025497,-0.011337
age,0.050840,0.062052,-0.116464,-0.052880,1.000000,-0.008444,0.104012,0.043066,0.128561,0.026617,-0.024340,0.054439,0.026750,-0.047480,0.015570,0.254252,0.023801,-0.148498,-0.057786,0.037795,0.015403,0.014040,0.030346,0.002236,0.045466,0.062557,0.008648,0.015553,0.004146,0.009797,0.007384,-0.001573,0.005359,-0.025394,-0.000382,0.002351,0.003162,-0.000122,-0.035260,-0.022551,0.009520,-0.003613,-0.014207,0.014467,0.052685
admission_type_id,-0.149827,-0.009516,-0.033249,0.011784,-0.008444,1.000000,0.095305,0.119555,-0.017495,-0.116115,0.134849,0.097352,0.038031,-0.017303,0.039435,-0.115241,0.340435,-0.048520,0.005744,-0.006430,-0.008813,0.006396,-0.002912,-0.002760,0.010921,-0.001508,0.001946,0.012086,0.012556,0.006057,-0.001561,0.002429,0.013609,-0.013677,-0.005955,-0.003526,-0.000371,0.002234,-0.005523,-0.008884,0.038790,-0.010256,0.063577,0.021656,0.023888
discharge_disposition_id,-0.138216,-0.142239,0.008626,-0.021852,0.104012,0.095305,1.000000,-0.000308,0.165358,0.017516,0.015288,0.110963,-0.017012,-0.026118,-0.024381,0.038946,0.025997,-0.013888,-0.000294,-0.005651,-0.011111,0.016459,-0.013635,0.013346,-0.004770,0.043370,0.001022,-0.013947,-0.002742,0.012572,0.005994,0.008139,0.001396,-0.021756,-0.005293,0.001537,0.000021,-0.000346,-0.003793,-0.019617,-0.032305,-0.017925,0.043621,0.000397,0.012179
admission_source_id,-0.112560,-0.018797,0.008553,0.003336,0.043066,0.119555,-0.000308,1.000000,0.007650,0.090763,-0.129703,-0.059007,0.018790,0.056924,0.029940,0.066291,0.372082,0.018901,-0.024816,-0.002338,-0.014910,0.000416,-0.019274,0.001229,0.014888,0.003331,0.003752,-0.004720,-0.006873,0.000459,0.